In [17]:
from tensorflow import keras as ks

(X_train, y_train), (X_test, y_test) = ks.datasets.imdb.load_data()
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [18]:
word_index = ks.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token

" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [19]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples

In [20]:
import tensorflow as tf

def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [21]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [22]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]
]

In [23]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))
# faaaaaantastic obtiene la posición 10053 porque no está en el vocabulario

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]], dtype=int64)>

In [24]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess).map(encode_words).prefetch(1)

embed_size = 128
model = ks.models.Sequential([
    ks.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None]),
    ks.layers.GRU(128, return_sequences=True),
    ks.layers.GRU(128),
    ks.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=100, callbacks=[ks.callbacks.EarlyStopping(monitor='accuracy', patience=10)])

Epoch 1/100
782/782 [==============================] - 144s 173ms/step - loss: 0.6072 - accuracy: 0.6390
Epoch 2/100
782/782 [==============================] - 116s 148ms/step - loss: 0.3861 - accuracy: 0.8293
Epoch 3/100
782/782 [==============================] - 116s 149ms/step - loss: 0.2324 - accuracy: 0.9111
Epoch 4/100
782/782 [==============================] - 127s 162ms/step - loss: 0.1404 - accuracy: 0.9499
Epoch 5/100
782/782 [==============================] - 128s 164ms/step - loss: 0.1112 - accuracy: 0.9610
Epoch 6/100
782/782 [==============================] - 137s 175ms/step - loss: 0.0892 - accuracy: 0.9691
Epoch 7/100
782/782 [==============================] - 136s 174ms/step - loss: 0.0644 - accuracy: 0.9773
Epoch 8/100
782/782 [==============================] - 136s 174ms/step - loss: 0.0491 - accuracy: 0.9836
Epoch 9/100
782/782 [==============================] - 136s 174ms/step - loss: 0.0468 - accuracy: 0.9840
Epoch 10/100
782/782 [==============================] -

In [33]:
import joblib

# save the model to disk
filename = 'sentiment100.pkl'
joblib.dump(model, filename)



['sentiment100.pkl']

In [34]:
# predict the sentiment of a review
def predict_sentiment(review, model):
    encoded_review = table.lookup(tf.constant([review.split()]))
    result = model.predict(tf.constant(encoded_review))[0][0]
    if result < 0.5:
        return "negative"
    else:
        return "positive"
    
predict_sentiment("I love this movie", model)

1/1 [==============================] - 0s 33ms/step


'positive'